## Importando as bibliotecas

In [1]:
import os
import numpy             as np
import pandas            as pd
import matplotlib.pyplot as plt
import seaborn           as sns

from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler, StandardScaler

%matplotlib inline
pd.set_option("display.max_columns", None)


## Definição de caminhos e diretórios

In [2]:
silver_path = "data/silver"

## Loading silver datasets

In [3]:
# Lista dos silvers
silver_list = os.listdir(f"{silver_path}")
file = silver_list[1]
df = pd.read_csv(f"{silver_path}/{file}")

df['Open_time'] = pd.to_datetime(df['Open_time'])
df['Close_time'] = pd.to_datetime(df['Close_time'])


In [4]:
print(df.isnull().sum())
df.dtypes
# df.head()

Open_time                       0
Open                            0
High                            0
Low                             0
Close                           0
Mean                            0
Volume                          0
Close_time                      0
Quote_asset_volume              0
Number_of_trades                0
Taker_buy_base_asset_volume     0
Taker_buy_quote_asset_volume    0
LT_SMA                          0
ST_SMA                          0
LT_EMA                          0
ST_EMA                          0
MACD                            0
Trigger_Line                    0
MADC_H                          0
UpperBand                       0
LowerBand                       0
RSI                             0
MFI                             0
ATR                             0
ForceIndex                      0
EMV                             0
dtype: int64


Open_time                       datetime64[ns]
Open                                   float64
High                                   float64
Low                                    float64
Close                                  float64
Mean                                   float64
Volume                                 float64
Close_time                      datetime64[ns]
Quote_asset_volume                     float64
Number_of_trades                         int64
Taker_buy_base_asset_volume            float64
Taker_buy_quote_asset_volume           float64
LT_SMA                                 float64
ST_SMA                                 float64
LT_EMA                                 float64
ST_EMA                                 float64
MACD                                   float64
Trigger_Line                           float64
MADC_H                                 float64
UpperBand                              float64
LowerBand                              float64
RSI          

#### Aplicanando Normalização.

In [5]:
def min_max_scaler( data, cols ):

    mm_scaler = MinMaxScaler()
    data[cols] = mm_scaler.fit_transform(np.array(data[cols]).reshape(-1,1))
    
    return data

# EDA

In [6]:
# Simple Moving Average 
def SMA(data, ndays, col, _name): 
    SMA = pd.Series(data[col].rolling(ndays).mean(), name = _name) 
    data = data.join(SMA) 
    return data

## Colorindo os pontos de MAX e MIN

In [7]:
def is_min(data, col, color_1 = 'lightblue'):
    
    if col+'_color' not in data.columns:
        data[col+'_color'] = color_1
        
    for i in range(2, len(data) - 2):
        if (data.loc[i, col] < data.loc[i - 1, col]) and (data.loc[i, col] < data.loc[i + 1, col]) and (data.loc[i, col] < data.loc[i - 2, col]) and (data.loc[i, col] < data.loc[i + 2, col]): 
            data.loc[i ,col+'_color'] = 'red'
            data.loc[i + 1, col+'_color'] = 'red'
            data.loc[i - 1, col+'_color'] = 'red'
    
    return data

def is_max(data, col, color_1 = 'lightblue'):
    
    if col+'_color' not in data.columns:
        data[col+'_color'] = color_1
        
    for i in range(2, len(data) - 2):
        if (data.loc[i, col] > data.loc[i - 1, col]) and (data.loc[i, col] > data.loc[i + 1, col]) and (data.loc[i, col] > data.loc[i - 2, col]) and (data.loc[i, col] > data.loc[i + 2, col]): 
            data.loc[i,col+'_color'] = 'green'
            data.loc[i - 1, col+'_color'] = 'green'
            data.loc[i + 1, col+'_color'] = 'green'
    
    return data

In [8]:

df = SMA(df, 5, 'Close', 'ST_SMA_AP')
df = is_min(df, 'ST_SMA_AP')
df = is_max(df, 'ST_SMA_AP')

df = SMA(df, 5, 'Meam', 'ST_SMA_Mean')
df = is_min(df, 'ST_SMA_Meam')
df = is_max(df, 'ST_SMA_Meam')

df = SMA(df, 5, 'RSI', 'ST_SMA_RSI')
df = is_min(df, 'ST_SMA_RSI')
df = is_max(df, 'ST_SMA_RSI')

df = SMA(df, 5, 'MFI', 'ST_SMA_MFI')
df = is_min(df, 'ST_SMA_MFI')
df = is_max(df, 'ST_SMA_MFI')

df = SMA(df, 5, 'ATR', 'ST_SMA_ATR')
df = is_min(df, 'ST_SMA_ATR')
df = is_max(df, 'ST_SMA_ATR')

df = SMA(df, 5, 'ForceIndex', 'ST_SMA_FI')
df = is_min(df, 'ST_SMA_FI')
df = is_max(df, 'ST_SMA_FI')

df = SMA(df, 5, 'EMV', 'ST_SMA_EMV')
df = is_min(df, 'ST_SMA_EMV')
df = is_max(df, 'ST_SMA_EMV')



df.head()

KeyError: 'Meam'

### Overview preço de fechamento

In [ ]:
# Difinindo datas de inicio e fim
min_date = pd.to_datetime('2022-01-01 00:00:00')
max_date = pd.to_datetime('2022-01-01 05:00:00')


# Aplicando o filtro para pegar somente as datas
df = df[(df['Close_time'] >= min_date) & (df['Close_time'] <= max_date)].copy()

# Definindo a figura para plotar
plt.figure(figsize=(20, 10))

# sns.scatterplot(x='Close_time', y='Close', data=df, c= df['ST_SMA_AP_color'])
sns.scatterplot(x = 'Close_time', y = 'ST_SMA_AP', data = df, c = df['ST_SMA_AP_color'])
sns.lineplot(x = 'Close_time', y = 'Close', data = df, color = 'blue')
plt.xlim(min_date, max_date)

plt.show()

## Relação entre preço de fechamento e indicadores

### Preço de fechamento vs RSI

In [ ]:
# Difinindo datas de inicio e fim
min_date = pd.to_datetime('2022-01-01 00:00:00')
max_date = pd.to_datetime('2022-01-01 05:00:00')

# Aplicando o filtro para pegar somente as datas
df_plot = df[(df['Close_time'] >= min_date) & (df['Close_time'] <= max_date)].copy()

df_plot = min_max_scaler(df_plot, 'Close')
df_plot = min_max_scaler(df_plot, 'RSI')
df_plot = min_max_scaler(df_plot, 'ST_SMA_RSI')

# Definindo a figura para plotar
plt.figure(figsize=(20, 8))
sns.lineplot(x = 'Close_time', y = 'Close', data = df_plot, color = 'blue')
sns.lineplot(x = 'Close_time', y = 'RSI', data = df_plot, color = 'orange')
sns.scatterplot(x = 'Close_time', y = 'ST_SMA_RSI', data = df_plot, c = df_plot['ST_SMA_RSI_color'])
plt.legend(['Close','RSI'])

plt.show()



#### Relação entre RSI e Preço de fechamento.
- Considerando um curto período. Se o preço e o RSI oscilam juntos, se um sobe o outro sobe se um desce o outro desce.   
- O RSI pode ter comportamento inverso do preço. Subir quando o preço está caindo ou cair quando o preço está subindo. Isso parace indicar um comportaemnto de inversão no preço.

### Preço de fechamento vs SMA

In [ ]:
# Difinindo datas de inicio e fim
min_date = pd.to_datetime('2022-01-01 00:00:00')
max_date = pd.to_datetime('2022-01-01 05:00:00')

# Aplicando o filtro para pegar somente as datas
df_plot = df[(df['Close_time'] >= min_date) & (df['Close_time'] <= max_date)]

# Definindo a figura para plotar 
plt.figure(figsize = (20,6))

sns.lineplot(x = 'Close_time', y = 'Close', data = df_plot, color = 'blue')
sns.lineplot(x = 'Close_time', y = 'ST_SMA', data = df_plot, color = 'green')
sns.lineplot(x = 'Close_time', y = 'LT_SMA', data = df_plot, color = 'orange')
plt.xlim(min_date, max_date)
plt.legend( ['Close','ST_SMA','LT_SMA'] )

plt.show()

#### Comportamento da SMA em relação ao comportamento do preço.   
- Se ST > LT indica um movimento de alta   
- SE ST < LT indidca um movimento de queda   
- As inversões no preço ocorrem próximos aos cruzamentos das SMA.   
- A diferença entre ST e LT aumenta, durante a queda, até o ponto de mínimo do preço.   
- A diferença entre ST e LT aumenta, durante a alta, até o ponto de  máximo do preço.   

### Preço de fechamento vs EMA

In [ ]:
# Difinindo datas de inicio e fim
min_date = pd.to_datetime('2022-01-01 00:00:00')
max_date = pd.to_datetime('2022-01-01 05:00:00')

# Aplicando o filtro para pegar somente as datas
df_plot = df[(df['Close_time'] >= min_date) & (df['Close_time'] <= max_date)]

# Definindo a figura para plotar 
plt.figure(figsize = (20,6))

sns.lineplot(x = 'Close_time', y = 'Close', data = df_plot, color = 'blue')
sns.lineplot(x = 'Close_time', y = 'ST_EMA', data = df_plot, color = 'green')
sns.lineplot(x = 'Close_time', y = 'LT_EMA', data = df_plot, color = 'orange')
plt.xlim(min_date, max_date)
plt.legend( ['Close','ST_EMA','LT_EMA'] )


plt.show()

#### Comportamento da EMA em relação ao comportamento do preço.   
- Se ST > LT indica um movimento de alta   
- SE ST < LT indidca um movimento de queda   
- As inversões no preço ocorrem próximos aos cruzamentos das EMA.   
- A diferença entre ST e LT aumenta, durante a queda, até o ponto de mínimo do preço.   
- A diferença entre ST e LT aumenta, durante a alta, até o ponto de  máximo do preço.   

### Preço de fechamento vs MACD

In [ ]:
# Difinindo datas de inicio e fim
min_date = pd.to_datetime('2022-01-01 00:00:00')
max_date = pd.to_datetime('2022-01-01 05:00:00')

# Aplicando o filtro para pegar somente as datas
df_plot = df[(df['Close_time'] >= min_date) & (df['Close_time'] <= max_date)]

# Definindo a figura para plotar 
plt.figure(figsize = (20,15))

plt.subplot(3,1,1)
sns.lineplot(x = 'Close_time', y = 'Close', data = df_plot, color = 'blue')
plt.xlim(min_date, max_date)

plt.subplot(3,1,2)
sns.lineplot(x = 'Close_time', y = 'MACD', data = df_plot, color = 'black')
sns.lineplot(x = 'Close_time', y = 'Trigger_Line', data = df_plot, color = 'red')
plt.xlim(min_date, max_date)
plt.legend( ['MACD','Trigger_Line'])

plt.subplot(3,1,3)
sns.barplot(x = 'Close_time', y = 'MADC_H', data = df_plot, color = 'orange')
plt.xticks(color = 'white' )

plt.show()

#### MACD_H em relação ao preço de fechamento.
- Valores positivos indicam movimentos de alta.   
- Valores negativos indicam movimentos de baixa.   
- Os pontos de máximo estão próximos ao preços máximos.
- Os pontos de mínimo estão próximos aos preços mínimos.

### Preço de fechamento vs MFI

In [ ]:
# Difinindo datas de inicio e fim
min_date = pd.to_datetime('2022-01-01 00:00:00')
max_date = pd.to_datetime('2022-01-01 05:00:00')

# Aplicando o filtro para pegar somente as datas
df_plot = df[(df['Close_time'] >= min_date) & (df['Close_time'] <= max_date)].copy()

df_plot = min_max_scaler(df_plot, 'Close')
df_plot = min_max_scaler(df_plot, 'MFI')
df_plot = min_max_scaler(df_plot, 'ST_SMA_MFI')


# Definindo a figura para plotar
plt.figure(figsize=(20, 8))
sns.lineplot(x = 'Close_time', y = 'Close', data = df_plot, color = 'blue')
sns.lineplot(x = 'Close_time', y = 'MFI', data = df_plot, color = 'orange')
sns.scatterplot(x = 'Close_time', y = 'ST_SMA_MFI', data = df_plot, c = df_plot['ST_SMA_MFI_color'])
plt.legend(['Close','MFI'])

plt.show()

#### MFI em função do preço médio
- Comportamento um pouco caótico. Deixar o modelo (BORUTA) decidir se é importante.

### Preço de fechamento vs ATR

In [ ]:
# Difinindo datas de inicio e fim
min_date = pd.to_datetime('2022-01-01 00:00:00')
max_date = pd.to_datetime('2022-01-01 05:00:00')

# Aplicando o filtro para pegar somente as datas
df_plot = df[(df['Close_time'] >= min_date) & (df['Close_time'] <= max_date)].copy()

df_plot = min_max_scaler(df_plot, 'Close')
df_plot = min_max_scaler(df_plot, 'ATR')
df_plot = min_max_scaler(df_plot, 'ST_SMA_ATR')


# Definindo a figura para plotar
plt.figure(figsize=(20, 8))
sns.lineplot(x = 'Close_time', y = 'Close', data = df_plot, color = 'blue')
sns.lineplot(x = 'Close_time', y = 'ATR', data = df_plot, color = 'orange')
sns.scatterplot(x = 'Close_time', y = 'ST_SMA_ATR', data = df_plot, c = df_plot['ST_SMA_ATR_color'])
plt.legend(['Close','ATR'])

plt.show()

#### ATR em função do preço médio
- Comportamento um pouco caótico. Deixar o modelo (BORUTA) decidir se é importante.

### Preço de fechamento vs Forceindex

In [ ]:
# Difinindo datas de inicio e fim
min_date = pd.to_datetime('2022-01-01 00:00:00')
max_date = pd.to_datetime('2022-01-01 05:00:00')

# Aplicando o filtro para pegar somente as datas
df_plot = df[(df['Close_time'] >= min_date) & (df['Close_time'] <= max_date)].copy()

df_plot = min_max_scaler(df_plot, 'Close')
df_plot = min_max_scaler(df_plot, 'ForceIndex')
df_plot = min_max_scaler(df_plot, 'ST_SMA_FI')


# Definindo a figura para plotar
plt.figure(figsize=(20, 8))
sns.lineplot(x = 'Close_time', y = 'Close', data = df_plot, color = 'blue')
sns.lineplot(x = 'Close_time', y = 'ForceIndex', data = df_plot, color = 'orange')
sns.scatterplot(x = 'Close_time', y = 'ST_SMA_FI', data = df_plot, c = df_plot['ST_SMA_FI_color'])
plt.legend(['Close','FI'])

plt.show()

#### ForceIndex em função do preço médio
- Comportamento bastante caótico. Deixar o modelo (BORUTA) decidir se é importante.   
- Provavelmente será descartado.

### Preço de fechamento vs EMV

In [ ]:
# Difinindo datas de inicio e fim
min_date = pd.to_datetime('2022-01-01 00:00:00')
max_date = pd.to_datetime('2022-01-01 05:00:00')

# Aplicando o filtro para pegar somente as datas
df_plot = df[(df['Close_time'] >= min_date) & (df['Close_time'] <= max_date)].copy()

df_plot = min_max_scaler(df_plot, 'Close')
df_plot = min_max_scaler(df_plot, 'EMV')
df_plot = min_max_scaler(df_plot, 'ST_SMA_EMV')


# Definindo a figura para plotar
plt.figure(figsize=(20, 8))
sns.lineplot(x = 'Close_time', y = 'Close', data = df_plot, color = 'blue')
sns.lineplot(x = 'Close_time', y = 'EMV', data = df_plot, color = 'orange')
sns.scatterplot(x = 'Close_time', y = 'ST_SMA_EMV', data = df_plot, c = df_plot['ST_SMA_EMV_color'])
plt.legend(['Close','EMV'])

plt.show()

### Overview preço Médio

In [ ]:
# Difinindo datas de inicio e fim
min_date = pd.to_datetime('2022-01-01 00:00:00')
max_date = pd.to_datetime('2022-01-01 05:00:00')


# Aplicando o filtro para pegar somente as datas
df = df[(df['Close_time'] >= min_date) & (df['Close_time'] <= max_date)].copy()

# Definindo a figura para plotar
plt.figure(figsize=(20, 10))

# sns.scatterplot(x='Close_time', y='Close', data=df, c= df['ST_SMA_AP_color'])
sns.scatterplot(x = 'Close_time', y = 'ST_SMA_Mean', data = df, c = df['ST_SMA_Mean_color'])
sns.lineplot(x = 'Close_time', y = 'Mean', data = df, color = 'blue')
plt.xlim(min_date, max_date)

plt.show()

## Relação entre preço Médio e indicadores

### Preço médio vs RSI

In [ ]:
# Difinindo datas de inicio e fim
min_date = pd.to_datetime('2022-01-01 00:00:00')
max_date = pd.to_datetime('2022-01-01 05:00:00')

# Aplicando o filtro para pegar somente as datas
df_plot = df[(df['Close_time'] >= min_date) & (df['Close_time'] <= max_date)].copy()

df_plot = min_max_scaler(df_plot, 'Close')
df_plot = min_max_scaler(df_plot, 'RSI')
df_plot = min_max_scaler(df_plot, 'ST_SMA_RSI')

# Definindo a figura para plotar
plt.figure(figsize=(20, 8))
sns.lineplot(x = 'Close_time', y = 'Close', data = df_plot, color = 'blue')
sns.lineplot(x = 'Close_time', y = 'RSI', data = df_plot, color = 'orange')
sns.scatterplot(x = 'Close_time', y = 'ST_SMA_RSI', data = df_plot, c = df_plot['ST_SMA_RSI_color'])
plt.legend(['Close','RSI'])

plt.show()



#### Relação entre RSI e Preço de fechamento.
- Considerando um curto período. Se o preço e o RSI oscilam juntos, se um sobe o outro sobe se um desce o outro desce.   
- O RSI pode ter comportamento inverso do preço. Subir quando o preço está caindo ou cair quando o preço está subindo. Isso parace indicar um comportaemnto de inversão no preço.

### Preço de fechamento vs SMA

In [ ]:
# Difinindo datas de inicio e fim
min_date = pd.to_datetime('2022-01-01 00:00:00')
max_date = pd.to_datetime('2022-01-01 05:00:00')

# Aplicando o filtro para pegar somente as datas
df_plot = df[(df['Close_time'] >= min_date) & (df['Close_time'] <= max_date)]

# Definindo a figura para plotar 
plt.figure(figsize = (20,6))

sns.lineplot(x = 'Close_time', y = 'Close', data = df_plot, color = 'blue')
sns.lineplot(x = 'Close_time', y = 'ST_SMA', data = df_plot, color = 'green')
sns.lineplot(x = 'Close_time', y = 'LT_SMA', data = df_plot, color = 'orange')
plt.xlim(min_date, max_date)
plt.legend( ['Close','ST_SMA','LT_SMA'] )

plt.show()

#### Comportamento da SMA em relação ao comportamento do preço.   
- Se ST > LT indica um movimento de alta   
- SE ST < LT indidca um movimento de queda   
- As inversões no preço ocorrem próximos aos cruzamentos das SMA.   
- A diferença entre ST e LT aumenta, durante a queda, até o ponto de mínimo do preço.   
- A diferença entre ST e LT aumenta, durante a alta, até o ponto de  máximo do preço.   

### Preço de fechamento vs EMA

In [ ]:
# Difinindo datas de inicio e fim
min_date = pd.to_datetime('2022-01-01 00:00:00')
max_date = pd.to_datetime('2022-01-01 05:00:00')

# Aplicando o filtro para pegar somente as datas
df_plot = df[(df['Close_time'] >= min_date) & (df['Close_time'] <= max_date)]

# Definindo a figura para plotar 
plt.figure(figsize = (20,6))

sns.lineplot(x = 'Close_time', y = 'Close', data = df_plot, color = 'blue')
sns.lineplot(x = 'Close_time', y = 'ST_EMA', data = df_plot, color = 'green')
sns.lineplot(x = 'Close_time', y = 'LT_EMA', data = df_plot, color = 'orange')
plt.xlim(min_date, max_date)
plt.legend( ['Close','ST_EMA','LT_EMA'] )


plt.show()

#### Comportamento da EMA em relação ao comportamento do preço.   
- Se ST > LT indica um movimento de alta   
- SE ST < LT indidca um movimento de queda   
- As inversões no preço ocorrem próximos aos cruzamentos das EMA.   
- A diferença entre ST e LT aumenta, durante a queda, até o ponto de mínimo do preço.   
- A diferença entre ST e LT aumenta, durante a alta, até o ponto de  máximo do preço.   

### Preço de fechamento vs MACD

In [ ]:
# Difinindo datas de inicio e fim
min_date = pd.to_datetime('2022-01-01 00:00:00')
max_date = pd.to_datetime('2022-01-01 05:00:00')

# Aplicando o filtro para pegar somente as datas
df_plot = df[(df['Close_time'] >= min_date) & (df['Close_time'] <= max_date)]

# Definindo a figura para plotar 
plt.figure(figsize = (20,15))

plt.subplot(3,1,1)
sns.lineplot(x = 'Close_time', y = 'Close', data = df_plot, color = 'blue')
plt.xlim(min_date, max_date)

plt.subplot(3,1,2)
sns.lineplot(x = 'Close_time', y = 'MACD', data = df_plot, color = 'black')
sns.lineplot(x = 'Close_time', y = 'Trigger_Line', data = df_plot, color = 'red')
plt.xlim(min_date, max_date)
plt.legend( ['MACD','Trigger_Line'])

plt.subplot(3,1,3)
sns.barplot(x = 'Close_time', y = 'MADC_H', data = df_plot, color = 'orange')
plt.xticks(color = 'white' )

plt.show()

#### MACD_H em relação ao preço de fechamento.
- Valores positivos indicam movimentos de alta.   
- Valores negativos indicam movimentos de baixa.   
- Os pontos de máximo estão próximos ao preços máximos.
- Os pontos de mínimo estão próximos aos preços mínimos.

### Preço de fechamento vs MFI

In [ ]:
# Difinindo datas de inicio e fim
min_date = pd.to_datetime('2022-01-01 00:00:00')
max_date = pd.to_datetime('2022-01-01 05:00:00')

# Aplicando o filtro para pegar somente as datas
df_plot = df[(df['Close_time'] >= min_date) & (df['Close_time'] <= max_date)].copy()

df_plot = min_max_scaler(df_plot, 'Close')
df_plot = min_max_scaler(df_plot, 'MFI')
df_plot = min_max_scaler(df_plot, 'ST_SMA_MFI')


# Definindo a figura para plotar
plt.figure(figsize=(20, 8))
sns.lineplot(x = 'Close_time', y = 'Close', data = df_plot, color = 'blue')
sns.lineplot(x = 'Close_time', y = 'MFI', data = df_plot, color = 'orange')
sns.scatterplot(x = 'Close_time', y = 'ST_SMA_MFI', data = df_plot, c = df_plot['ST_SMA_MFI_color'])
plt.legend(['Close','MFI'])

plt.show()

#### MFI em função do preço médio
- Comportamento um pouco caótico. Deixar o modelo (BORUTA) decidir se é importante.

### Preço de fechamento vs ATR

In [ ]:
# Difinindo datas de inicio e fim
min_date = pd.to_datetime('2022-01-01 00:00:00')
max_date = pd.to_datetime('2022-01-01 05:00:00')

# Aplicando o filtro para pegar somente as datas
df_plot = df[(df['Close_time'] >= min_date) & (df['Close_time'] <= max_date)].copy()

df_plot = min_max_scaler(df_plot, 'Close')
df_plot = min_max_scaler(df_plot, 'ATR')
df_plot = min_max_scaler(df_plot, 'ST_SMA_ATR')


# Definindo a figura para plotar
plt.figure(figsize=(20, 8))
sns.lineplot(x = 'Close_time', y = 'Close', data = df_plot, color = 'blue')
sns.lineplot(x = 'Close_time', y = 'ATR', data = df_plot, color = 'orange')
sns.scatterplot(x = 'Close_time', y = 'ST_SMA_ATR', data = df_plot, c = df_plot['ST_SMA_ATR_color'])
plt.legend(['Close','ATR'])

plt.show()

#### ATR em função do preço médio
- Comportamento um pouco caótico. Deixar o modelo (BORUTA) decidir se é importante.

### Preço de fechamento vs Forceindex

In [ ]:
# Difinindo datas de inicio e fim
min_date = pd.to_datetime('2022-01-01 00:00:00')
max_date = pd.to_datetime('2022-01-01 05:00:00')

# Aplicando o filtro para pegar somente as datas
df_plot = df[(df['Close_time'] >= min_date) & (df['Close_time'] <= max_date)].copy()

df_plot = min_max_scaler(df_plot, 'Close')
df_plot = min_max_scaler(df_plot, 'ForceIndex')
df_plot = min_max_scaler(df_plot, 'ST_SMA_FI')


# Definindo a figura para plotar
plt.figure(figsize=(20, 8))
sns.lineplot(x = 'Close_time', y = 'Close', data = df_plot, color = 'blue')
sns.lineplot(x = 'Close_time', y = 'ForceIndex', data = df_plot, color = 'orange')
sns.scatterplot(x = 'Close_time', y = 'ST_SMA_FI', data = df_plot, c = df_plot['ST_SMA_FI_color'])
plt.legend(['Close','FI'])

plt.show()

#### ForceIndex em função do preço médio
- Comportamento bastante caótico. Deixar o modelo (BORUTA) decidir se é importante.   
- Provavelmente será descartado.

### Preço de fechamento vs EMV

In [ ]:
# Difinindo datas de inicio e fim
min_date = pd.to_datetime('2022-01-01 00:00:00')
max_date = pd.to_datetime('2022-01-01 05:00:00')

# Aplicando o filtro para pegar somente as datas
df_plot = df[(df['Close_time'] >= min_date) & (df['Close_time'] <= max_date)].copy()

df_plot = min_max_scaler(df_plot, 'Close')
df_plot = min_max_scaler(df_plot, 'EMV')
df_plot = min_max_scaler(df_plot, 'ST_SMA_EMV')


# Definindo a figura para plotar
plt.figure(figsize=(20, 8))
sns.lineplot(x = 'Close_time', y = 'Close', data = df_plot, color = 'blue')
sns.lineplot(x = 'Close_time', y = 'EMV', data = df_plot, color = 'orange')
sns.scatterplot(x = 'Close_time', y = 'ST_SMA_EMV', data = df_plot, c = df_plot['ST_SMA_EMV_color'])
plt.legend(['Close','EMV'])

plt.show()

#### EMV em função do preço médio
- Os máximos e os mínimos do EMV parecem coincidir com os máximos e os mínimos do preço de fechamento.

#### EMV em função do preço médio
- Os máximos e os mínimos do EMV parecem coincidir com os máximos e os mínimos do preço de fechamento.


### Relação entre inversão de comportamento do preço e os indicadores dos a dois. Ocorre cruzamento?



### Estimar quais indicadores melhor indicam inversão de comportamento.

# Featrue Engineering


### Diferença entre LT_SMA e ST_SMA 

### Diferença entre LT_EWMA e ST_EWMA

### sinal da inclinação para o preço e para todos os indicadores.

### Seno e Cosseno dos osciladores

#### gerar três diferentes grupos de dados:
* -5 (compra)   
* 0 (aguarda)   
* 5 (vende) 


# Treinamento do modelo   

### Três estratégias:   
1 - Classificar os dados em Compra, vende ou espera

2 - Prever se o preço sobe ou desce

3 - Prever o preço em análise de curto espaço de tempo.

Para os três casos rodar o Biruta e registrar experimentos no ML Flow.


### Estratégias 1: Classificadores e agrupadores



#### 1 - Kmeans 



#### 2 - DB Scan



#### 3 - KNearst Neighboor



#### 4 - Decision Tree



#### 5 - Random Forest



#### 6 - XGBoost Classifier



### Estratégia 2 Classificadores binários



#### Logístic Regressivo 



#### KNearst Neighboor



#### Decision Tree 



#### Random Forest



#### XGBoost Classifier



### Estratégia 3 Regressão.



#### Regressão linear



#### Regressão Linear Lasso ou Ride



#### Randon Forest



#### XGBosst Regressor

# Avaliação da Performance do modelo



# Finetuning (Hyper parâmetros)



### Random Search



### Ridige Search



### Zielzera Search




# Reavaliação dos modelos.


                          
# RELATÓRIO TOP.